In [1]:
import pandas as  pd
import os
import xml.etree.ElementTree as ET

## ***PARTIE 1*** ##

In [8]:
dfG = pd.read_csv("Data/GSM3533230_CGND-HRA-00013_counts.txt", sep="\t")
dfG.head()

,gene/TE,../../STAR_v252_mapping/CGND_12616/HRA-00013_PE.bam
0,A1BG,42
1,A1BG-AS1,149
2,A1CF,0
3,A2M,3327
4,A2M-AS1,101


In [9]:
def emplacement(l,gene_name):
    for i in range(len(l)):
        if gene_name == l[i]:
            return i+1


In [10]:
class MyDataFrame:
    
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.df_list = [] # liste pour stocker les dataframes
        self.df = pd.DataFrame() # dataframe final
        
        # liste de tous les fichiers dans le dossier
        file_list = os.listdir(self.folder_path)

        # boucle à travers tous les fichiers dans le dossier
        for filename in file_list:
            if filename.endswith(".txt"): # vérifie que le fichier est un fichier .txt
                file_path = os.path.join(self.folder_path, filename) 
                df = pd.read_csv(file_path, sep="\t")  # crée un dataframe à partir du fichier
                self.df_list.append(df) # ajoute le dataframe à la liste

        self.df = pd.concat(self.df_list, axis=1) # concatène tous les dataframes dans la liste en un seul dataframe à partir de la colonne 1
        self.df = self.df.loc[:,~self.df.columns.duplicated()] # supprime les colonnes "gene/TE" en doublon
        self.df = self.df.T
    
    #affichage
    def aff(self):
        print(self.df)
        
    # Getter pour accéder à la valeur d'une cellule en donnant le nom de l'échantillon et le nom du gène en paramètre
    def get_value(self, sample_id, gene_name):
        return self.df[sample_id][emplacement(self.df.iloc[0].values,gene_name)]
    
    def get_colonne(self,gene_name):
        return self.df.iloc[emplacement(self.df.iloc[0].values,gene_name)].values
    def get_ligne(self,id):
        return self.df.iloc[id].values
    def getMatrix(self):
        return self.df
    




In [11]:
mat = MyDataFrame("Data")

mat.df
#m.iloc[0]

,0,1,2,3,4,5,6,7,8,9,...,28943,28944,28945,28946,28947,28948,28949,28950,28951,28952
gene/TE,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,X6B_LINE:CR1:LINE,X7A_LINE:CR1:LINE,X7B_LINE:CR1:LINE,X7C_LINE:CR1:LINE,X7D_LINE:CR1:LINE,X8_LINE:CR1:LINE,X9_LINE:L1:LINE,Zaphod2:hAT-Tip100:DNA,Zaphod3:hAT-Tip100:DNA,Zaphod:hAT-Tip100:DNA
../../STAR_v252_mapping/CGND_12952/HRA-00756_PE.bam,42,189,0,4052,92,201,1,8,76,1,...,1023,2363,2137,1265,100,607,140,551,2269,2233
../../STAR_v252_mapping/CGND_12616/HRA-00064_PE.bam,37,110,0,4660,102,64,7,1,68,0,...,880,2077,1908,1072,119,478,129,525,1821,1921
../../STAR_v252_mapping/CGND_12966/HRA-00421_PE.bam,31,110,0,1484,84,60,1,1,12,0,...,656,1357,1345,749,61,341,102,346,1122,1437
../../STAR_v252_mapping/CGND_12922/HRA-00358_PE.bam,14,41,0,2185,105,198,4,0,24,0,...,958,2017,1817,1087,99,527,145,446,1522,1757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
../../BAM/CGND_13028/HRA-00252_PE.bam,29,142,0,1429,102,115,2,4,14,1,...,671,1526,1319,735,85,394,92,330,1279,1408
../../BAM/CGND_13043/HRA-00529_PE.bam,31,142,0,1526,108,122,2,1,139,0,...,758,1724,1575,866,76,427,122,430,1478,1660
../../STAR_v252_mapping/CGND_12966/HRA-00424_PE.bam,39,95,0,1223,74,89,1,3,39,0,...,687,1424,1340,693,101,354,83,300,1145,1390
../../BAM/CGND_13132/HRA-01269_PE.bam,46,179,0,2899,89,109,1,6,46,1,...,796,1694,1488,869,89,469,99,442,1684,1836


In [16]:
# LECTURE FICHIER XML
url = '{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}'
tree = ET.parse('Data/GSE124439_family.xml')
root = tree.getroot()
#samples = root.findall(url+"Sample")
#print(samples)

data_list = []

# Loop to loop through samples and extract other annotations
for sample in root.findall(url+'Sample'): 
    #Sample ID retrieval
    #print(sample)
    sample_id = sample.get('iid')
    
    # Recovery of annotations
    annotations = {}
    for i in sample.findall(url+'Channel'):
        #print(i)
        for charac in i.findall(url+'Characteristics'):
            if charac.get('tag') in ['cns subregion', 'subject id', 'sample group']:
                annotations[charac.get('tag')] = charac.text.strip()    
    # Add data to list
    data = {'sample_id': sample_id, **annotations}
    data_list.append(data)
#print(ET.tostring(root, encoding='unicode'))    
# Creating the DataFrame from the data

df = pd.DataFrame(data_list)  

In [17]:
print(df)

      sample_id   cns subregion   subject id      sample group
0    GSM3533230  Frontal Cortex  NEUEL133AK6  ALS Spectrum MND
1    GSM3533231  Frontal Cortex  NEUYV496XLP  ALS Spectrum MND
2    GSM3533232  Frontal Cortex  NEUZU200WEQ  ALS Spectrum MND
3    GSM3533233  Frontal Cortex  NEUME287RK2  ALS Spectrum MND
4    GSM3533234  Frontal Cortex  NEUEC006FND  ALS Spectrum MND
..          ...             ...          ...               ...
171  GSM3533401  Frontal Cortex  NEUMY028JH4  ALS Spectrum MND
172  GSM3533402  Frontal Cortex  NEUXR145UBL  ALS Spectrum MND
173  GSM3533403  Frontal Cortex  NEUKN209FNW  ALS Spectrum MND
174  GSM3533404  Frontal Cortex  NEUUF289NRL  ALS Spectrum MND
175  GSM3533405  Frontal Cortex  NEUPU334DL1  ALS Spectrum MND

[176 rows x 4 columns]
